## *Natural reading task*
# Visualizations

In [1]:
%matplotlib ipympl
import os.path as op
import numpy as np
import mne
from mne.datasets import fetch_fsaverage
from mayavi import mlab
import matplotlib.pyplot as plt

In [2]:
# Set up dirs needed for all visualizations
experiment_dir = '../natural_reading/'

## Sensor space

In [4]:
# Read average evoked file
evoked_dir = op.join(experiment_dir, 'evoked')
evoked_file =  op.join(evoked_dir, 'average-ave.fif')
average_evoked = mne.read_evokeds(evoked_file, condition = 0, baseline=None, proj=False,).crop(-0.050,None)

Reading ../natural_reading/evoked/average-ave.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 96)  idle
    Found the data of interest:
        t =    -250.00 ...     600.00 ms (Grand average (n = 39))
        0 CTF compensation matrices available
        nave = 39 - aspect type = 100
No baseline correction applied


In [23]:
# Plot evoked potentials 
fig0 = average_evoked.plot(spatial_colors=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# Plot only in one time window 
average_evoked.crop(0.0,0.450) # change time-window here
fig1 = average_evoked.plot(spatial_colors=False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Source space

### Time-line 

In [ ]:
noise_cov = 'with_non_reg_noise_cov'
method = 'MNE'
views = ['caudal', 'lateral']
hemis = ['both', 'lh'] # looped together with views, as a tuple

# Select time points to show
time_points = np.linspace(0.060, 0.130, 8)
time_sections = [0,2,4,6] # index of first subplot in figure, 2 images are shown per figure

mlab.options.offscreen = True

average_stc_file = op.join(experiment_dir, noise_cov, method, 'average')
average_stc = mne.read_source_estimate(average_stc_file, 'fsaverage')

for t in time_sections:
    for view, hemi in zip(views, hemis):
        fig1, axs1 = plt.subplots(1,2, sharex=True, sharey = True)
#         fig1.suptitle(condition.capitalize() + ' condition')
        surfer_kwargs = dict(hemi=hemi, clim=dict(kind='value', lims=[6.5e-12, 7.0e-12, 1.6e-11 ]),
                             views=view, time_unit='s', smoothing_steps=5, colorbar = False, size = 500, time_label = None)
        for col,time_point in enumerate(time_points[t:(t+2)]):
            brain = average_stc.plot(initial_time = time_point, **surfer_kwargs)
            img = mlab.screenshot()
            mlab.close()
            axs1[col].imshow(img)
            axs1[col].set_axis_off()
            axs1[col].set_title(str(int(time_point*1000))+ ' ms')


### Interactive figure with time viewer

In [27]:
method = 'eLORETA'
# lims = [2, 5, 31 ]
noise_cov = 'with_non_reg_noise_cov'
views = ['lat','caudal','ventral']
mlab.options.offscreen = False

stc_file = op.join(experiment_dir, noise_cov, method, 'average')
stc = mne.read_source_estimate(stc_file, 'fsaverage')
tview_kwargs = dict(hemi='split', #clim=dict(kind='value', lims=lims),
                     views=views, time_unit='s', smoothing_steps=5, colorbar = True, size = 1400, 
                     initial_time = 0.0, time_viewer = True)
brain = stc.plot(**tview_kwargs)

Using control points [0.25363966 0.30715286 1.06028318]


### Create and save video

In [ ]:
tmin = 0.220
tmax = 0.300
method = 'MNE'
lims = [6.5e-12, 7.0e-12, 1.6e-11 ]
noise_cov = 'with_non_reg_noise_cov'
views = ['lat','caudal','ventral']
video_file = 'NR_P2_'+ method +'.mov'

mlab.options.offscreen = False
stc_file = op.join(experiment_dir,noise_cov, method,'average')
stc = mne.read_source_estimate(stc_file, 'fsaverage')
video_kwargs = dict(hemi='split', clim=dict(kind='value', lims=lims),
                     views=views, time_unit='s', smoothing_steps=5, colorbar = True, size = 1600)
brain = stc.plot(**video_kwargs)
brain.save_movie(video_file, tmin=tmin, tmax= tmax, time_dilation = 200, framerate = 24)
mlab.close()

## Cross-hemisphere contrast in source space

In [18]:
# Contrast function
def create_cross_hemi_contrast(stc, tmin = None, tmax = None):
    
    # Read files if input is string
    if isinstance(stc, str):
        stc = mne.read_source_estimate(stc, 'fsaverage')

    # Crop
    stc.crop(tmin, tmax)
    
    # Load FsAverage anatomy
    fs_dir = fetch_fsaverage(verbose=False)
    subjects_dir = op.dirname(fs_dir)
    
    # Morph the data to fsaverage_sym, for which we have left_right registrations:
    stc = mne.compute_source_morph(stc,'fsaverage', 'fsaverage_sym', smooth=5, warn=True,
                                   subjects_dir=subjects_dir).apply(stc)
    # Compute a morph-matrix mapping the right to the left hemisphere,
    # and vice-versa.
    morph = mne.compute_source_morph(stc, 'fsaverage_sym', 'fsaverage_sym',
                                     spacing=stc.vertices, warn=False,
                                     subjects_dir=subjects_dir, xhemi=True,
                                     verbose='error')  # creating morph map
    stc_xhemi = morph.apply(stc)

    # Create contrast
    contrast = stc - stc_xhemi

    # Return stc with contrast data
    return contrast

### Interactive figure with time-viewer

In [28]:
tmin = 0.0
tmax = 0.450
method = 'eLORETA'
noise_cov = 'with_non_reg_noise_cov'
# STC files
stc_file = op.join(experiment_dir, noise_cov, method, 'average')

contrast = create_cross_hemi_contrast(stc_file, tmin = tmin, tmax = tmax) # create contrast stc

lims = [1.5e-12, 1.5e-12, 6.5e-12]

mlab.options.offscreen = False
contrast_kwargs = dict(hemi='split', views=['lateral', 'caudal', 'ventral'], #clim=dict(kind='value', pos_lims=lims),
                       initial_time= tmin, time_unit='s', size=(1200, 1200), smoothing_steps=5, time_viewer = True)

brain = contrast.plot(**contrast_kwargs)

Using control points [0.05290117 0.06464495 0.24552155]
